In [6]:
!cp origin/vimm7.odt origin/vimm7a.odt
srcStem0='origin/vimm7'
srcStem='origin/vimm7a'

import zipfile, re, subprocess
def getOdfFile(odf,what,mode='r'):
    with zipfile.Path(odf,at=what).open(mode=mode) as item: return item.read()
def setOdfFile(odf,what,data):
    assert isinstance(data,(str,bytes))
    # TODO (?): copy, modify rename back and overwrite
    subprocess.call(['zip','--quiet','--delete',odf,what])
    with zipfile.ZipFile(odf,'a',compression=zipfile.ZIP_DEFLATED) as odf2:
        with odf2.open(what,'w') as content:
            content.write(data if isinstance(data,bytes) else data.encode('utf-8'))
from lxml import etree
for stem in srcStem,srcStem0:
    for what in ['content','styles']:
        tree=etree.fromstring(getOdfFile(stem+'.odt',what=f'{what}.xml',mode='rb'),etree.XMLParser())
        open(f'{stem}.{what}.xml','w').write(etree.tostring(tree,encoding='unicode',pretty_print=True))


In [7]:
if 0:
    from lxml import etree
    from rich.pretty import pprint
    # dta=getOdfFile(srcStem+'.odt',what='content.xml',mode='rb')
    dta=open(srcStem+'.content.xml').read()
    vimm=etree.fromstring(dta,etree.XMLParser())
    textNs='{'+vimm.nsmap['text']+'}'
    ns=dict(namespaces=vimm.nsmap)
    def _(t): return f'"{"" if t is None else t}"'
    for para in vimm.xpath('.//text:p[@text:style-name!="VimmExportSkip"]',**ns):
        for s in para.xpath(f'.//text:span[starts-with(@text:style-name,"Vimm")]',**ns):
            if len(s)==0: continue
            sty=s.attrib[textNs+'style-name']
            for ss in s:
                if len(ss)>0:
                    print(sty,s.attrib[textNs+'style-name'],s.sourceline)
                    continue
                if s.text is None: s.text=(ss.text if ss.text else '')
                else: s.text+=(ss.text if ss.text else '')
                s.text+=(ss.tail if ss.tail else '')
                s.remove(ss)
    
        
    open(srcStem+'.content.xml','w').write(etree.tostring(vimm,encoding='unicode',pretty_print=True))
    setOdfFile(srcStem+'.odt',what='content.xml',data=etree.tostring(vimm,pretty_print=False))
    
if 0:
    # check paragraphs with unknown style in the main text (removed manually in LO)
    from lxml import etree
    from rich.pretty import pprint
    #dta=getOdfFile(srcStem+'.odt',what='content.xml',mode='rb')
    dta=open(srcStem+'.content.xml').read()
    vimm=etree.fromstring(dta,etree.XMLParser())
    textNs='{'+vimm.nsmap['text']+'}'
    ns=dict(namespaces=vimm.nsmap)
    def _(t): return f'"{"" if t is None else t}"'
    def _tag(e): return e.tag.split('}')[-1]
    def _style(e): return e.attrib[textNs+'style-name']
    txt=vimm.xpath('.//office:text',**ns)[0]
    print(txt)
    for para in txt:
        if _tag(para)=='sequence-decls': continue
        # if _tag(para) not in ('p','h','list'):
        style=_style(para)
        if style=='VimmExportSkip': continue
        if not style.startswith('Vimm') and _tag(para)!='h':
            print(_tag(para),para.sourceline,style)

if 1:
    # remove irrelevant styles
    def _(t): return ('' if t is None else t)
    def _tag(e): return e.tag.split('}')[-1]
    def _style(e): return e.attrib[textNs+'style-name']
    from lxml import etree
    from rich.pretty import pprint
    #dta=getOdfFile(srcStem+'.odt',what='content.xml',mode='rb')
    dta=open(srcStem+'.content.xml').read()
    vimm=etree.fromstring(dta,etree.XMLParser())
    textNs='{'+vimm.nsmap['text']+'}'
    styleNs='{'+vimm.nsmap['style']+'}'
    officeoooNs='{'+vimm.nsmap['officeooo']+'}'
    foNs='{'+vimm.nsmap['fo']+'}'
    ns=dict(namespaces=vimm.nsmap)
    styles=vimm.xpath('.//office:automatic-styles',**ns)[0]
    irrelevant=set()
    for sty in styles:
        if sty.get(styleNs+'family','None')!='text': continue
        if len(sty)==0: continue
        c1=sty[0]
        if sum([1 for attr in (foNs+'language',officeoooNs+'rsid',styleNs+'language-asian') if attr in c1.attrib])>0:
            irrelevant.add(sty.attrib[styleNs+'name'])
    print(irrelevant)
    # actually consider all T* styles as irrelevant
    # also, de-nest nested VimmPali and similar
    for span in vimm.xpath('.//text:span',**ns):
        style=span.get(textNs+'style-name',None)
        prev,parent=span.getprevious(),span.getparent()
        parentStyle=parent.get(textNs+'style-name',None)
        if style!=parentStyle and not style.startswith('T'): continue
        if prev is not None: prev.tail=_(prev.tail)+_(span.text)+_(span.tail)
        else:
            assert parent.index(span)==0
            parent.text=_(parent.text)+_(span.text)+_(span.tail)
        # add children nodes
        for c in span: span.addprevious(c)
        parent.remove(span)
        
    pMap=dict([(s.attrib[styleNs+'name'],s.attrib[styleNs+'parent-style-name']) for s in styles if (s.get(styleNs+'family',None)=='paragraph' and styleNs+'parent-style-name' in s.attrib)])
    for p in vimm.xpath('.//text:p',**ns):
        sty=_style(p)
        if sty in pMap: p.attrib[textNs+'style-name']=pMap[sty]
        
    open(srcStem+'1.content.xml','w').write(etree.tostring(vimm,encoding='unicode',pretty_print=True))
    
    while True:
        replacements=0
        for span in vimm.xpath('.//text:span',**ns):
            if span.tail is not None: continue
            if (span2:=span.getnext()) is None: continue
            sty=span.get(textNs+'style-name',None)
            sty2=span2.get(textNs+'style-name',None)
            if sty!=sty2: continue
            #assert len(span)==0
            #assert len(span2)==0
            if len(span)!=0 or len(span2)!=0:
                print(span.sourceline,sty,sty2)
                continue
            replacements+=1
            span.text=_(span.text)+_(span2.text)
            assert span.tail is None
            span.tail=span2.tail
            span2.getparent().remove(span2)
        if replacements==0: break
    for span in vimm.xpath('.//text:span',**ns):
        assert len(span)==0
        #if len(span)>0:
        #    sty=span.get(textNs+'style-name',None)
        #    print(span.sourceline,span.tag,sty,', '.join([s.get(textNs+'style-name','XXX') for s in span]))
            
    open(srcStem+'2.content.xml','w').write(etree.tostring(vimm,encoding='unicode',pretty_print=True))
        
        
    open(srcStem+'.content.xml','w').write(etree.tostring(vimm,encoding='unicode',pretty_print=True))
    setOdfFile(srcStem+'.odt',what='content.xml',data=etree.tostring(vimm,pretty_print=False))     
    

        
        
    open(srcStem+'.content.xml','w').write(etree.tostring(vimm,encoding='unicode',pretty_print=True))
    setOdfFile(srcStem+'.odt',what='content.xml',data=etree.tostring(vimm,pretty_print=False))

{'T31', 'T24', 'T37', 'T28', 'T9', 'T33', 'T35', 'T23', 'T27', 'T14', 'T10', 'T20', 'T22', 'T21', 'T7', 'T17', 'T18', 'T25', 'T8', 'T38', 'T16', 'T30', 'T11', 'T19', 'T26', 'T13', 'T36', 'T29', 'T15', 'T34', 'T12', 'T32'}
